<a href="https://colab.research.google.com/github/lx-jdar/progra-concurrente/blob/development/tp3/java/tp3_java.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo práctico 1 - Parte 3: Java

Creación del Archivo Java

In [5]:
%%writefile App.java
import java.util.ArrayList;
import java.util.HashMap;
import java.util.Map;
import java.util.Optional;
import java.util.Random;
import java.util.concurrent.ArrayBlockingQueue;

public class App
{

  private static final int MIN_VALUE = 0;
  private static final int MAX_VALUE = 99;
  private static int quantity;

  public static class Producer extends Thread
  {
    private final ArrayBlockingQueue<Optional<Integer>> queue;

    public Producer(ArrayBlockingQueue<Optional<Integer>> queue)
    {
      this.queue = queue;
    }

    public void run()
    {
      try
      {
        for (int i = 0; i < quantity; i++)
        {
          // Genero un valor random
          Random random = new Random();
          int item = random.nextInt(MAX_VALUE - MIN_VALUE + 1) + MIN_VALUE;
          // Agrego el valor a la cola
          queue.put(Optional.ofNullable(item));
          Thread.sleep(100);
        }
        // Agrego null para avisar al consumidor que el productor dejo de producir
        queue.put(Optional.ofNullable(null));
      }
      catch (InterruptedException e)
      {
        Thread.currentThread().interrupt();
      }
    }
  }

  public static class Consumer extends Thread
  {
    private final ArrayBlockingQueue<Optional<Integer>> queue;
    private ArrayList<Integer> values = new ArrayList<>();

    public Consumer(ArrayBlockingQueue<Optional<Integer>> queue)
    {
      this.queue = queue;
    }

    public void run()
    {
      try
      {
        // Tomo valor de la cola
        Optional<Integer> value = queue.take();
        // El isPresent() me indica que existe el valor y no es null
        while (value.isPresent())
        {
          // Guardo el valor
          values.add(value.get());
          Thread.sleep(200);
          value = queue.take();
        }
      }
      catch (InterruptedException e)
      {
        Thread.currentThread().interrupt();
      }
      this.showStadistics();
    }

    private void showStadistics()
    {
      int sum = 0;
      int min = MAX_VALUE;
      int max = MIN_VALUE;

      for (int value : values)
      {
        sum += value;
        min = Math.min(min, value);
        max = Math.max(max, value);
      }

      double average = (double) sum / values.size();

      System.out.println("Sum: " + sum);
      System.out.println("Average: " + average);
      System.out.println("Min: " + min);
      System.out.println("Max: " + max);
      System.out.print("Most Frequent Values: ");
      getMostFrequentValues().forEach((value) -> System.out.print(value + " "));
      System.out.println();
    }

    private ArrayList<Integer> getMostFrequentValues ()
    {
      Map<Integer, Integer> mapValues = new HashMap<>();

        // Genero el map con los valores y la cantidad de veces que aparecen
      for (Integer value : values)
      {
        mapValues.put(value, mapValues.getOrDefault(value, 0) + 1);
      }

      // Busco cual es la maxima cantidad
      int maxCount = 0;
      for (int count : mapValues.values())
      {
        maxCount = Math.max(maxCount, count);
      }

      // Busco todos los valores con la max cantidad
      ArrayList<Integer> mostFrequentNumbers = new ArrayList<>();
      for (Map.Entry<Integer, Integer> entry : mapValues.entrySet())
      {
        if (entry.getValue() == maxCount)
        {
          mostFrequentNumbers.add(entry.getKey());
        }
      }

      return mostFrequentNumbers;
    }
  }

  public static void main(String[] args) throws Exception
  {
    try
    {
      quantity = Integer.parseInt(args[0]);
    }
    catch (NumberFormatException e)
    {
      // En caso que el valor pasado por argumento no sea un numero
      System.err.println("Invalid input. Please provide a valid integer as the first argument.");
      System.exit(-1);
    }
    catch (ArrayIndexOutOfBoundsException e)
    {
      // En caso que no se haya pasado ningun argumento
      System.err.println("No input provided. Please provide an integer as the first argument.");
      System.exit(-1);
    }

    // Creo el ArrayBlockingQueue
    ArrayBlockingQueue<Optional<Integer>> queue = new ArrayBlockingQueue<>(quantity);

    System.out.println("Loading...");

    // Creo el productor y el consumidor
    Thread producerThread = new Thread(new Producer(queue));
    Thread consumerThread = new Thread(new Consumer(queue));

    producerThread.start();
    consumerThread.start();

    producerThread.join();
    consumerThread.join();
  }
}



Writing App.java


Compilacion del Archivo java

In [6]:
!javac App.java

Ejecución de distintos ejemplos del Programa en Consola

In [7]:
!java App

No input provided. Please provide an integer as the first argument.


In [8]:
!java App 10

Loading...
Sum: 456
Average: 45.6
Min: 20
Max: 90
Most Frequent Values: 34 67 20 52 22 71 26 90 44 30 


In [9]:
!java App 15

Loading...
Sum: 952
Average: 63.46666666666667
Min: 21
Max: 98
Most Frequent Values: 98 55 46 


# Comentarios

> El desarrollo en Java para manejo de concurrencia no es facil. Se necesita contar con conocimiento específico de la sintaxis del lenguaje. Tiene una fuerte programación orientada a objetos, lo que lo hace el desarrollo un poco mas tardía, pero con mayor alcance quizás en relación a otros lenguajes. Es fuertemente tipado, lo cual conduce a declarar cada objeto que será utilizado en ambito determinado. Por otra parte tiene mecanimos propias del lenguaje(como ArrayBlockingQueue), acomplados especificamente para la comunicadión entre threads. Los tipos especificos, dan como resultado una mejor organización del código, que luego se buildea en conjunto.



# Conclusion

> Se puede concluir que Java cuenta con la ayuda de la comunidad para la programación, con lo cual es facil de recolectar información para que se desarrolle un programa. Sin embargo una vez inmerso en el lenguaje, se vuelve facil de adaptar para cualquier tipo de tarea. Quizás no es el lenguaje mas performante para algunas tareas como la concurrencia, pero tiene un amplio manejo de distintas utilidades.

